In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sys
sys.path.insert(0, '../automin/autodiff')
import variables as v
import AD_numpy as anp
import vector_variables as vv
sys.path.insert(0, '../automin')
import optimizer as op

# An Optimization Package with AD functionalities

## Motivation

### Motivation

Taking derivatives is an essential operation in numerical methods, optimization, and science. From a computational perspective, however, calculating a derivative can be difficult.

**Finite Differences** requires careful selection of $\epsilon$

**Symbolic Differentiation** is infeasible for complicated functions, especially for higher order derivatives

### Motivation

**Automatic Differentiation** overcomes these challenges by providing both quick and accurate derivatives.

This also allows us to provide optimization methods that are fast and accurate. Most of the standard optimization modules such as `scipy.optimize` do not rely on automatic differentiation. 

### Brief Mathematical Background 

...

### Demo for AD module

In [36]:
# user has a given function
f1 = lambda x, y: anp.exp(3*x) + anp.log(y/x)

In [37]:
# he can calculate its value
f1(2, 3)

403.83425860084327

In [38]:
# but now he wishes to get its derivative
a = v.Variable('a', 2)
b = v.Variable('b', 3)
res1 = f1(a,b)

In [5]:
print('Value: {}.'.format(res1.val))
print('Partial Derivative wrt a: {}'.format(res1.partial_der(a)))
print('Jacobian: {}'.format(res1.jacobian()))

Value: 403.83425860084327.
Partial Derivative wrt a: 1209.7863804782053
Jacobian: {'b': 0.3333333333333333, 'a': 1209.7863804782053}


### Demo for AD module

In [39]:
# similarly, if he has a vector function
@vv.vectorize_variable
def vec_fn(x, y, z):
    f1 = x * y + anp.sin(y) + anp.cos(z)
    f2 = x + y + anp.sin(x*y)
    return np.array([f1,f2])

In [40]:
# he can extract the values, jacobian, and partial derivatives similarly
c = v.Variable('c', 2)
res2 = vec_fn(a, b, c)

In [41]:
print('Value: {}.'.format(res2.val))
print('Partial Derivative wrt a: {}'.format(res2.partial_der(a)))
display(res2.jacobian())

Value: [3.42532415 4.14112001].
Partial Derivative wrt a: [1.        0.0100075]


,a,b,c
0,1.000000,3.540302,-0.909297
1,0.010008,-1.969977,0.000000


### Now all that seems trivial. 

When would a user just want to solely calculate the gradient of a function?

#### Instead, suppose a user wants to find the root of a given function using Newton's method

In [26]:
def newton_method_scalar(fn, x, threshold, max_iter, verbose=True, norm=2):
    
    var_names = ['x'+str(idx) for idx in range(len(x))]
    
    x = np.array(x)
    nums_iteration = 0
    while True:
        x_new = x - fn(*x) / _get_grad(fn, x, var_names)

        # print iteration output
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {x} ')
        
        # threshold stopping condition 
        if np.linalg.norm(x-x_new, norm) < threshold:
            print(f'After {nums_iteration} iterations, found a root: {x_new}')
            break
        
        # iteration stopping condition
        if nums_iteration >= max_iter:
            break
        nums_iteration +=1
        x = x_new
        
def _get_grad(fn, x, var_names):
    variables = [v.Variable(var_names[idx], x_n) for idx, x_n in enumerate(x)]
    out = fn(*variables)
    jacobian = out.jacobian()
    grad = np.array([jacobian[name] for name in var_names])
    return grad

### Newton's Method

In [28]:
f2 = lambda x, y, z: (x-4)**2 + (y-3)**2 + (z-2)**2
newton_method_scalar(f2, [3, 2, 1], 1e-6, 50, verbose=True)

Iteration at 0, at [3 2 1] 
Iteration at 1, at [4.5 3.5 2.5] 
Iteration at 2, at [3.75 2.75 1.75] 
Iteration at 3, at [4.125 3.125 2.125] 
Iteration at 4, at [3.9375 2.9375 1.9375] 
Iteration at 5, at [4.03125 3.03125 2.03125] 
Iteration at 6, at [3.984375 2.984375 1.984375] 
Iteration at 7, at [4.0078125 3.0078125 2.0078125] 
Iteration at 8, at [3.99609375 2.99609375 1.99609375] 
Iteration at 9, at [4.00195312 3.00195312 2.00195312] 
Iteration at 10, at [3.99902344 2.99902344 1.99902344] 
Iteration at 11, at [4.00048828 3.00048828 2.00048828] 
Iteration at 12, at [3.99975586 2.99975586 1.99975586] 
Iteration at 13, at [4.00012207 3.00012207 2.00012207] 
Iteration at 14, at [3.99993896 2.99993896 1.99993896] 
Iteration at 15, at [4.00003052 3.00003052 2.00003052] 
Iteration at 16, at [3.99998474 2.99998474 1.99998474] 
Iteration at 17, at [4.00000763 3.00000763 2.00000763] 
Iteration at 18, at [3.99999619 2.99999619 1.99999619] 
Iteration at 19, at [4.00000191 3.00000191 2.00000191] 
I

**It is relatively simple to write up code for algorithms requiring first-order derivatives!**

**A common class of problems requiring derivatives is optimization**

As such, we decided to use our AD module as a basis for an optimization package!

### Optimization

Generally, optimization is to find the 'best available' values of an objective function given a defined domain

**Simply, optimization can be thought of as maximizing (or minimizing) a real function**

Our optimization module thus seeks to provide users a way to minimize functions. We have implemented 4 first-order iterative algorithms to minimize a function.

### Optimization

1. Gradient Descent

2. Steepest Descent

3. Conjugate Gradient 

4. Broyden-Fletcher-Goldfarb-Shanno (BFGS) 

### Gradient Descent

Gradient descent is the simplest first-order iterative algorithm but introduces a key idea that forms the basis of iterative algorithms. 

**Notice that if we walk in the negative direction of the gradient, it seems like we will eventually reach the minimum**

### Gradient Descent

Mathematically, for a function $\ell(x)$ this is saying that to move from $x^{(t)}$ to $x^{(t+1)}$, we should follow the path of steepest descent $-\nabla\ell(x^{(t)})$

If we do this over and over again ...

![](figs/gradient-descent1.png)

![](figs/gradient-descent2.png)

![](figs/gradient-descent3.png)

### It is clear that we will eventually reach the minimum!

### However, an even better method would be to update the direction with each step

This will allow us to pick favorable directions that take better steps or to pick 'non-interfering' directions

### Steepest Descent

Steepest descent chooses a new direction through a line minimzation along a direction $s$, before moving along a new direction $-\nabla f$. 

![steepestdescent.png](figs/steepestdescent.png)

### This basically means that steepest descent is picking a new direction that is orthogonal to the previous direction


![descent.png](figs/descent.png)

### Conjugate Gradient

After the line minimization along a direction $u$, instead of moving along a direction $v = -\nabla f$ which is the direction of steepest descent, we can instead perturb the direction to ensure that $\nabla f$ is perpendicular to the line minimization direction $u$ before and after we move along the perturbed direction.

![](figs/conjugategradient.jpg)

**This ensures that the conjugate gradient method moves in non-interfering directions**

In steepest descent, we only make right angle turns. Each step is along directions that can undo the minimization in the previous step. As such, we might take some time before it reaches the minimum. This is often the case in practice.

Conjugate gradient instead uses conjugate directions which preserve the minimization achieved in the previous step. 

![](figs/descent.png)

### BFGS



### Future Work

- Extract Hessian of Jacobian matrices, and 2nd order derivatives to expand optimization methods to include methods such as Newton's method, etc.
- Matrix calculus with applications in calculating gradients of complex deep learning networks for loss minimization. Could also be useful in mixture models, such as when using EM algorithm to analyze Gaussian Mixture models.
